# Wrangling data...

In order to use your marketing data correctly it needs to be in the right format to run calculations, draw charts and marke predictions. Gettting your data in the correct order is refered to as data wrangling. 

In the section you will:

<ul>
    <li>Correcting data from Google Analytics by creating a new page URL</li>
    <li>Group data</li>
    <li>Fix data types</li>
    <li>Drop columns</li>
    <li>Replace data values</li>
    <li>Join two dataset to create a new dataset</li>
    <li>Save new datasets as CSV</li>
</ul>

In [2]:
import pandas as pd # Lodad the pandas library

In [37]:
#Import google analytics pages data
ga_page_data = pd.read_csv('C:/Users/ysman/Documents/springboard/Timeseries Python Linkedin/Ex_Files_Python_Marketing/Ex_Files_Python_Marketing/Exercise Files/data/ga_data/ga_pages.csv', skiprows=6, nrows=376)
ga_page_traffic_data = pd.read_csv('C:/Users/ysman/Documents/springboard/Timeseries Python Linkedin/Ex_Files_Python_Marketing/Ex_Files_Python_Marketing/Exercise Files/data/ga_data/ga_pages.csv', 
                                   skiprows=386, nrows=366, parse_dates=['Day Index'])

In [4]:
#Importing pages data from Google Search Console
gsc_page = pd.read_csv('C:/Users/ysman/Documents/springboard/Timeseries Python Linkedin/Ex_Files_Python_Marketing/Ex_Files_Python_Marketing/Exercise Files/data/gsc_data/gsc_pages.csv')

In [6]:
#Import Facebook ad data
fb_data = pd.read_csv('C:/Users/ysman/Documents/springboard/Timeseries Python Linkedin/Ex_Files_Python_Marketing/Ex_Files_Python_Marketing/Exercise Files/data/other_data/facebook-data-ads.csv', 
                      parse_dates=['Reporting starts','Reporting ends'])

## Fix Google Analytics Page data

We know that some of the data being gathered in Google Analytics is wrong. Google Analytics has been gathering homepage data under two page, these pages are `/` and `/www.miratrix.co.uk`.

Also during this process we will want to create a URL that can be paired with data gathered from Google Search Console. I'll split the data on '/' to allow me to break the URL into three parts. '/' denotes a breaks in the URL. Then I will rebuild the URL in the order which matches the GSC data URL's.

In [7]:
#Look at Google Analytics data
ga_page_data.head()

,Page,Pageviews,Unique Pageviews,Avg. Time on Page,Entrances,Bounce Rate,% Exit,Page Value
0,/,813,665,00:02:05,641,14.35%,55.97%,£0.00
1,/www.miratrix.co.uk,515,388,00:02:17,373,59.25%,56.89%,£0.00
2,/get-in-touch/,336,163,00:01:06,23,10.53%,33.04%,£0.00
3,/app-marketing-agency/,140,120,00:02:04,73,18.31%,57.86%,£0.00
4,/app-store-optimization-services/,136,105,00:01:30,22,14.29%,47.79%,£0.00


### Recreate Google Analytics Page URL

In [9]:
#Split the url by /
split_dom = ga_page_data.Page.str.rpartition("/")

In [10]:
split_dom.head()

,0,1,2
0,,/,
1,,/,www.miratrix.co.uk
2,/get-in-touch,/,
3,/app-marketing-agency,/,
4,/app-store-optimization-services,/,


In [11]:
split_dom[2].replace("", "miratrix.co.uk", inplace=True)

In [12]:
#Change URLformat so that it matches other data sources 
split_dom[2].replace("www.miratrix.co.uk", "miratrix.co.uk", inplace=True)

In [13]:
split_dom.head()

,0,1,2
0,,/,miratrix.co.uk
1,,/,miratrix.co.uk
2,/get-in-touch,/,miratrix.co.uk
3,/app-marketing-agency,/,miratrix.co.uk
4,/app-store-optimization-services,/,miratrix.co.uk


In [14]:
#Build the URL and replace current URL's with new URL format
ga_page_data.Page = "https://" + split_dom[2] + split_dom[0] + "/"

In [15]:
ga_page_data.head()

,Page,Pageviews,Unique Pageviews,Avg. Time on Page,Entrances,Bounce Rate,% Exit,Page Value
0,https://miratrix.co.uk/,813,665,00:02:05,641,14.35%,55.97%,£0.00
1,https://miratrix.co.uk/,515,388,00:02:17,373,59.25%,56.89%,£0.00
2,https://miratrix.co.uk/get-in-touch/,336,163,00:01:06,23,10.53%,33.04%,£0.00
3,https://miratrix.co.uk/app-marketing-agency/,140,120,00:02:04,73,18.31%,57.86%,£0.00
4,https://miratrix.co.uk/app-store-optimization-...,136,105,00:01:30,22,14.29%,47.79%,£0.00


### Grouping Data

In order to fix the issue of having two homepage URL (and possibly more) we will need to group the data by Page. This poses some issues as not all the data are numerical and you are also dealing with a timeseries.

What can be done is to change the object fields to numerical and to convert the time on page column to a timedelta.

In [16]:
ga_page_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 376 entries, 0 to 375
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Page               376 non-null    object
 1   Pageviews          376 non-null    int64 
 2   Unique Pageviews   376 non-null    int64 
 3   Avg. Time on Page  376 non-null    object
 4   Entrances          376 non-null    int64 
 5   Bounce Rate        376 non-null    object
 6   % Exit             376 non-null    object
 7   Page Value         376 non-null    object
dtypes: int64(3), object(5)
memory usage: 23.6+ KB


#### Wrangling data types for Groupby function

In [17]:
#Replace 0  with time format 00:00:00
ga_page_data['Avg. Time on Page'] = ga_page_data['Avg. Time on Page'].replace(0, '00:00:00')

In [18]:
#Convert column from string to time delta
ga_page_data['Avg. Time on Page'] = pd.to_timedelta(ga_page_data['Avg. Time on Page'])

In [19]:
#Replace % with nothing
ga_page_data['Bounce Rate'] = ga_page_data['Bounce Rate'].str.replace('%',"")
#Convert the column into a Float
ga_page_data['Bounce Rate'] = ga_page_data['Bounce Rate'].astype('float')

In [20]:
#Replace % with nothing
ga_page_data['% Exit'] = ga_page_data['% Exit'].str.replace('%',"")
#Convert the column into a Float
ga_page_data['% Exit'] = ga_page_data['% Exit'].astype('float')

In [21]:
#Replace % with nothing
ga_page_data['Page Value'] = ga_page_data['Page Value'].str.replace('£',"")
#Convert the column into a Float
ga_page_data['Page Value'] = ga_page_data['Page Value'].astype('float')

In [22]:
#Set the index for the dataframe to Page
ga_page_data.set_index('Page',inplace=True)

In [23]:
#Check the data to make sure the data types have changed
ga_page_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 376 entries, https://miratrix.co.uk/ to https://www.miratrix.co.uk#utm_source=nav&utm_medium=traffic&utm_campaign=datatools/
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype          
---  ------             --------------  -----          
 0   Pageviews          376 non-null    int64          
 1   Unique Pageviews   376 non-null    int64          
 2   Avg. Time on Page  376 non-null    timedelta64[ns]
 3   Entrances          376 non-null    int64          
 4   Bounce Rate        376 non-null    float64        
 5   % Exit             376 non-null    float64        
 6   Page Value         376 non-null    float64        
dtypes: float64(3), int64(3), timedelta64[ns](1)
memory usage: 23.5+ KB


#### Applying Groupby to the data

In [24]:
#Groupby Index and get the mean of the 'Avg. Time on Page'
avg_time = ga_page_data.groupby(ga_page_data.index)['Avg. Time on Page'].mean(numeric_only=False)

In [25]:
#Convert avg_time to dataframe and store as ga_Time
ga_time = pd.DataFrame(avg_time)

In [26]:
#Groupby bounce, exit, pageviews and Unique pageviews using the appropriate math 
#and store in a dataframe. Save as ga_bou_ex_type
ga_bou_ex_type = pd.DataFrame(ga_page_data.groupby(ga_page_data.index).agg({
    'Bounce Rate': ['mean'], "% Exit": ['mean'],'Pageviews': sum, 'Unique Pageviews': sum}))

In [30]:
#Check your time groupby
ga_time['Avg. Time on Page'].head()

Page
https://#utm_source=nav&utm_medium=traffic&utm_campaign=datatools/                                                  0 days 00:00:19
https://#utm_source=nav&utm_medium=traffic&utm_campaign=datatools/app-marketing-agency/                             0 days 00:00:04
https://+&cd=10&hl=en&ct=clnk&gl=in/search?q=cache:bI4Yk20ZHV8J:https://miratrix.co.uk/get-in-touch/                0 days 00:00:00
https://+&cd=5&hl=en&ct=clnk&gl=uk/search?q=cache:CAP7TiQpKL4J:https://miratrix.co.uk/tag/bridgnorth-shoplifters/   0 days 00:00:00
https://39/blog/page/                                                                                               0 days 00:00:00
Name: Avg. Time on Page, dtype: timedelta64[ns]

In [31]:
#Check your aggreated groupby
ga_bou_ex_type.info()

<class 'pandas.core.frame.DataFrame'>
Index: 316 entries, https://#utm_source=nav&utm_medium=traffic&utm_campaign=datatools/ to https://xFRAx2jfTOmg+6EPY+09e3SpRN05tZm6PA=&b=5&f=frame&u=quh9O6fk3cp+g1Q0B84ejuJAQEZJll4HdKCQfOVJedw=&b=5/go.php?u=quh9O7m+gcpnkEcjR8wG07lIAEwDjhYPfuuLYf1GaJpICjY4ebQy5itrwJGI8mdy/xFTBhuneDOmg+6EPY+09e3SpRN05tZo5/w=&b=5&f=frame&u=quh9O7m+gcpnkEcjR8wG07lIAEwDjhYPfuuLYf1GaJpICjY4ebQy5itrwJGI8mdy/
Data columns (total 4 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   (Bounce Rate, mean)      316 non-null    float64
 1   (% Exit, mean)           316 non-null    float64
 2   (Pageviews, sum)         316 non-null    int64  
 3   (Unique Pageviews, sum)  316 non-null    int64  
dtypes: float64(2), int64(2)
memory usage: 12.3+ KB


#### Rebuilding the Google Analytics dataframe

Here we will rebuild the Google Analytics dataset with the fixed data for use later.

In [32]:
#Merge ga_time and ga_bou_ex_type on page and with an inner join
new_ga = pd.merge(ga_time, #left dataset
                  ga_bou_ex_type, #right dataset
                  how='inner', # we're joning using the data in the column
                  left_on="Page", # the left dataset is neing joined on the column Page
                  right_on="Page" # the right dataset is neing joined on the column Page
                 ).sort_values(by="Avg. Time on Page" ,ascending=False) #sort the new dataframe

C:\Users\ysman\anaconda3\lib\site-packages\pandas\core\reshape\merge.py:643: UserWarning: merging between different levels can give an unintended result (1 levels on the left,2 on the right)
  warnings.warn(msg, UserWarning)


In [33]:
new_ga.head()

,Avg. Time on Page,"(Bounce Rate, mean)","(% Exit, mean)","(Pageviews, sum)","(Unique Pageviews, sum)"
Page,,,,,
https://channels/channel/UCAQfRNzXGD4BQICkO1KQZUA/,0 days 00:21:45,0.0,0.0,1,1
https://miratrix.co.uk/top-3-competitor-analysis-for-aso/,0 days 00:18:09.500000,50.0,87.5,5,5
https://www.miratrix.co.uk?SuperSocializerAuth=LiveJournal/blog/page/30/,0 days 00:14:57,0.0,0.0,1,1
https://about/channel/UCAQfRNzXGD4BQICkO1KQZUA/,0 days 00:14:45,0.0,0.0,2,2
https://videos?view=0&sort=p&flow=grid/channel/UCAQfRNzXGD4BQICkO1KQZUA/,0 days 00:14:43,0.0,0.0,1,1


In [34]:
#rename (Bounce Rate, mean), (% Exit, mean), (Pageviews, sum), (Unique Pageviews, sum)
new_ga = new_ga.rename(columns={('Bounce Rate', 'mean'): 'Bounce Rate', ('% Exit', 'mean') : 'Exit',
                               ('Pageviews', 'sum') : 'Pageviews', 
                               ('Unique Pageviews', 'sum') : 'Unique Pageviews'})

In [35]:
#Check the rename worked
new_ga.head()

,Avg. Time on Page,Bounce Rate,Exit,Pageviews,Unique Pageviews
Page,,,,,
https://channels/channel/UCAQfRNzXGD4BQICkO1KQZUA/,0 days 00:21:45,0.0,0.0,1,1
https://miratrix.co.uk/top-3-competitor-analysis-for-aso/,0 days 00:18:09.500000,50.0,87.5,5,5
https://www.miratrix.co.uk?SuperSocializerAuth=LiveJournal/blog/page/30/,0 days 00:14:57,0.0,0.0,1,1
https://about/channel/UCAQfRNzXGD4BQICkO1KQZUA/,0 days 00:14:45,0.0,0.0,2,2
https://videos?view=0&sort=p&flow=grid/channel/UCAQfRNzXGD4BQICkO1KQZUA/,0 days 00:14:43,0.0,0.0,1,1


## Dropping Columns

Sometimes we end up with data that we don't need that we want to drop out of our dataframe. In this tutuorial you'll learn how to drop columns from Google Analytics and Facebook ads data.

In [38]:
#Check the Google Analytics Page data
ga_page_traffic_data.head()

,Day Index,Pageviews,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7
0,2018-08-31,14,NaN,NaN,NaN,NaN,NaN,NaN
1,2018-09-01,6,NaN,NaN,NaN,NaN,NaN,NaN
2,2018-09-02,17,NaN,NaN,NaN,NaN,NaN,NaN
3,2018-09-03,14,NaN,NaN,NaN,NaN,NaN,NaN
4,2018-09-04,11,NaN,NaN,NaN,NaN,NaN,NaN


In [41]:
#Drop NaN values from the dataset
ga_page_traffic_data.dropna(axis=1, inplace=True)

In [42]:
#Check the dropna worked
ga_page_traffic_data.head()

,Day Index,Pageviews
0,2018-08-31,14
1,2018-09-01,6
2,2018-09-02,17
3,2018-09-03,14
4,2018-09-04,11


In [40]:
#Check the Facebook Ads data
fb_data.head()

,Unnamed: 0,Reporting starts,Reporting ends,Campaign name,Delivery,Mobile app installs,Mobile app actions,Cost per mobile app install (GBP),Cost per mobile app action (GBP),Desktop app installs,Desktop app engagement,Cost per desktop app install (GBP),Cost per desktop app engagement (GBP),Post reactions,Post comments,Post shares
0,0,2014-02-21,2018-01-29,Campaign1,0,11451.0,NaN,1.275683,NaN,NaN,NaN,NaN,NaN,1807.0,42.0,133.0
1,1,2015-12-04,2015-12-04,Campaign2,inactive,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN
2,2,2015-10-21,2015-10-21,Campaign3,inactive,2.0,NaN,3.530000,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN
3,3,2015-10-20,2015-10-20,Campaign3,inactive,2.0,NaN,16.500000,NaN,NaN,NaN,NaN,NaN,15.0,NaN,NaN
4,4,2015-10-20,2015-10-20,Campaign4,inactive,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [43]:
#Drop all columns that have NaN
fb_data.drop('Unnamed: 0', axis=1, inplace=True)

In [44]:
#Check the drop worked
fb_data.head()

,Reporting starts,Reporting ends,Campaign name,Delivery,Mobile app installs,Mobile app actions,Cost per mobile app install (GBP),Cost per mobile app action (GBP),Desktop app installs,Desktop app engagement,Cost per desktop app install (GBP),Cost per desktop app engagement (GBP),Post reactions,Post comments,Post shares
0,2014-02-21,2018-01-29,Campaign1,0,11451.0,NaN,1.275683,NaN,NaN,NaN,NaN,NaN,1807.0,42.0,133.0
1,2015-12-04,2015-12-04,Campaign2,inactive,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN
2,2015-10-21,2015-10-21,Campaign3,inactive,2.0,NaN,3.530000,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN
3,2015-10-20,2015-10-20,Campaign3,inactive,2.0,NaN,16.500000,NaN,NaN,NaN,NaN,NaN,15.0,NaN,NaN
4,2015-10-20,2015-10-20,Campaign4,inactive,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Replacing missing (NaN) Facebook Ad data 

We cans see from the Facebook Ad data that there are a lot of NaN (Not A Number) values, these values are missing data. In order to visualise this data we will need to replace this data with a number. In this case we will use 0.

In [45]:
fb_data.fillna(0,inplace=True)

In [46]:
#Check that fillna worked
fb_data.head()

,Reporting starts,Reporting ends,Campaign name,Delivery,Mobile app installs,Mobile app actions,Cost per mobile app install (GBP),Cost per mobile app action (GBP),Desktop app installs,Desktop app engagement,Cost per desktop app install (GBP),Cost per desktop app engagement (GBP),Post reactions,Post comments,Post shares
0,2014-02-21,2018-01-29,Campaign1,0,11451.0,0.0,1.275683,0.0,0.0,0.0,0.0,0.0,1807.0,42.0,133.0
1,2015-12-04,2015-12-04,Campaign2,inactive,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
2,2015-10-21,2015-10-21,Campaign3,inactive,2.0,0.0,3.530000,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
3,2015-10-20,2015-10-20,Campaign3,inactive,2.0,0.0,16.500000,0.0,0.0,0.0,0.0,0.0,15.0,0.0,0.0
4,2015-10-20,2015-10-20,Campaign4,inactive,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Joining Google Analytics and Google Search Console Data

In order to extract more value from your datasets you can join them together so that you can view the trends all in one place. Pandas has various methods for joining data in this case you will use `pd.merge`.

In [47]:
#You pandas merge to join Google Console and Google Analytics data
ga_gsc_data = pd.merge(gsc_page, #left data
                       new_ga, #right data
                       how='inner', 
                       left_on="Page", 
                       right_index=True)

In [48]:
#Check it worked
ga_gsc_data.head()

,Page,Clicks,Impressions,CTR,Position,Avg. Time on Page,Bounce Rate,Exit,Pageviews,Unique Pageviews
0,https://miratrix.co.uk/,371,100673,0.37%,42.59,0 days 00:01:43.666666666,47.296667,54.286667,1396,1095
1,https://miratrix.co.uk/app-marketing-agency/,80,30824,0.26%,26.84,0 days 00:01:45.333333333,64.436667,60.913333,210,181
2,https://miratrix.co.uk/mobile-app-competitor-a...,16,1458,1.1%,30.64,0 days 00:04:06,53.335000,89.475000,30,26
3,https://miratrix.co.uk/blippar-the-message-isn...,11,517,2.13%,36.05,0 days 00:00:22.333333333,58.170000,78.333333,27,21
4,https://miratrix.co.uk/author/nduddy/,10,140,7.14%,5.61,0 days 00:02:52,22.223333,38.890000,15,13


## Saving your data to a CSV 

Saving data in pandas is super easy. We'll being using .to_csv() to save the augement and new datasets for use later.

In [49]:
#Save the newly created Page dataset
ga_gsc_data.to_csv('C:/Users/ysman/Documents/springboard/Timeseries Python Linkedin/Ex_Files_Python_Marketing/Ex_Files_Python_Marketing/Exercise Files/data/other_data/ga_gsc_data.csv')

In [50]:
#Save the amended facebook dataset
fb_data.to_csv('C:/Users/ysman/Documents/springboard/Timeseries Python Linkedin/Ex_Files_Python_Marketing/Ex_Files_Python_Marketing/Exercise Files/data/other_data/fixed_fb_data.csv')